In [ ]:
import pandas as pd
#xác định kích thước của mỗi toán hạng trong một công thức hoành chỉnh
def sizeElement(listSizeEle, sizeEle, total_size):
  # print(sizeEle)
  if sum(sizeEle) == total_size:
    if len(sizeEle) % 2 == 1:
      sizeEle.append(0)
      listSizeEle.append(sizeEle.copy())
      sizeEle.pop()
    else:
      listSizeEle.append(sizeEle.copy())
  else:
    if len(sizeEle) == 0:
      for i in range(1, total_size - sum(sizeEle)+1):
        sizeEle.append(i)
        sizeElement(listSizeEle, sizeEle, total_size)
        sizeEle.pop()
    else:
      if len(sizeEle) % 2 == 1:
        for i in range(total_size - sum(sizeEle)+1):
          sizeEle.append(i)
          sizeElement(listSizeEle, sizeEle, total_size)
          sizeEle.pop()
      else:
        for i in range(1, total_size - sum(sizeEle)+1):
          sizeEle.append(i)
          sizeElement(listSizeEle, sizeEle, total_size)
          sizeEle.pop()
  return listSizeEle


#sắp xếp loại bỏ các tập kích thước trùng nhau
def processListSize(listSizeEle):
  result = []
  for i in range(len(listSizeEle)):
    listSizeEle[i] = [(listSizeEle[i][j], listSizeEle[i][j+1]) for j in range(0 , len(listSizeEle[i]), 2)]
    listSizeEle[i].sort(key=lambda x : x[0])
    listSizeEle[i].sort(key=lambda x : x[0]+x[1])

  dict_list = {}
  for i in range(len(listSizeEle)):
    try:
      k = dict_list[str(listSizeEle[i])]
    except:
      dict_list[str(listSizeEle[i])] = 1
      result.append(listSizeEle[i])

  for i in range(len(result)):
    dict_ele_size = {}
    for j in range(len(result[i])):
      try:
          dict_ele_size[f"{result[i][j][0]}_{result[i][j][1]}"] += 1
      except:
          dict_ele_size[f"{result[i][j][0]}_{result[i][j][1]}"] = 1
    result[i] = dict_ele_size
  return result

In [ ]:
result = []
element = ""
index = []
last_op = ""
#đưa vào danh sách các toán hạng vào số lần lặp của nó
def crElement(elements, times):
  global result, element, index, last_op
  if len(index) == times:
    result.append(element)
  else:
    if len(index) == 0:
      start = 0
    else:
      start = index[-1] 
    last_op_psedou = last_op
    for i in range(start, len(elements)):
      for op in ['+', '-']:
        if i == start and op != last_op and last_op != "":
          continue
        else:
          element += op + elements[i]
          index.append(i)
          last_op = op
          crElement(elements, times)
          element = element[:len(element)-len(elements[0])-1]
          index.pop()
          last_op = last_op_psedou


In [ ]:
index_file = 0
def ghepcongthuc(list_cong_thuc_con, index):
  global index_file
  list_cong_thuc = [list_cong_thuc_con[0]]
  while len(list_cong_thuc) < len(list_cong_thuc_con)- 1:
    n = len(list_cong_thuc)
    list_cong_thuc.append([])
    for i in range(len(list_cong_thuc[-2])):
      for j in range(len(list_cong_thuc_con[n])):
        list_cong_thuc[-1].append(list_cong_thuc[-2][i] + list_cong_thuc_con[n][j])
  list_cong_thuc.append([])
  count = 0
  for i in range(len(list_cong_thuc[-2])):
    for j in range(len(list_cong_thuc_con[-1])):
      list_cong_thuc[-1].append(list_cong_thuc[-2][i] + list_cong_thuc_con[-1][j])
      count += 1
      if count == int(50000000):
        count = 0
        df = pd.DataFrame({"congthuc":list_cong_thuc[-1]})
        df.to_csv(f'drive/MyDrive/congthuc_{index}bien/{index}bien_{index_file}.csv.gz', sep='|', header=True, index=False, chunksize=100000, compression='gzip', encoding='utf-8')
        df = 0
        list_cong_thuc[-1] = []
        index_file += 1
  if count != 0:
    df = pd.DataFrame({"congthuc":list_cong_thuc[-1]})
    df.to_csv(f'drive/MyDrive/congthuc_{index}bien/{index}bien_{index_file}.csv.gz', sep='|', header=True, index=False, chunksize=100000, compression='gzip', encoding='utf-8')
    list_cong_thuc = []
    df = 0
    index_file += 1
  list_cong_thuc_con = 0

In [ ]:
#list_exp_psedou[-1] : là list công thức sau mỗi lần lặp
def crExp(n):
  global result, element, index, last_op, index_file
  #có thể chia nhỏ list_size ra để sinh công thức
  list_size = processListSize(sizeElement([], [], n))
  list_exp_child = []
  data_file = []
  count = 1
  for i in range(len(list_size)):
    for key, value in list_size[i].items():
      crElement(list(pd.read_csv(f"drive/MyDrive/sinh_loc_cong_thuc/congthuc/congthuc{key}.csv")['congthuc']), value)
      list_exp_child.append(result.copy())
      count *= len(result)
      result = []
      element = ""
      index = []
      last_op = ""
    result = []
    if count > 50000000:
      ghepcongthuc(list_exp_child, n)
      list_exp_child = []  
    else:
      list_exp_psedou = [list_exp_child[0]]
      for t in range(1, len(list_exp_child)):
        list_exp_psedou.append([])
        for j in range(len(list_exp_child[t])):
          for k in range(len(list_exp_psedou[-2])):
            list_exp_psedou[-1].append(list_exp_psedou[-2][k]+list_exp_child[t][j])
      list_exp_child = []      
      df = pd.DataFrame({"congthuc":list_exp_psedou[-1]})
      df.to_csv(f'drive/MyDrive/congthuc_{n}bien/{n}bien_{index_file}.csv.gz', sep='|', header=True, index=False, chunksize=100000, compression='gzip', encoding='utf-8')
      df = 0
      list_exp_psedou = 0
      index_file += 1
    print(f"DONE {i}")
    count = 1
  return "DONE"
print(crExp(8))

In [ ]:
#lưu file
'''
import pandas as pd
def expInvalid(exp1, exp2):
  key = dict()
  for i in range(0, len(exp1), 2):
    key[exp1[i]] = 1
  for i in range(0, len(exp2), 2):
    try:
      key[exp2[i]] += 1
      return False
    except:
      pass
  return True

print(expInvalid("ba", "bcd"))
def saveFile(n):
  key = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'I':9}
  dp = "ABCDEFGHI"
  exp_multi = [list(dp)]
  exp_divide = []
  while len(exp_multi) <  n:
    exp_multi.append([])
    for i in range(len(exp_multi[-2])):
      for j in range(key[exp_multi[-2][i][-1]] - 1, len(dp)):
        exp_multi[-1].append(exp_multi[-2][i]+"*"+dp[j])
  for i in range(len(exp_multi)):
    exp_divide.append(list(map(lambda x : x.replace("*", "/"), exp_multi[i])))
  result = []
  size = []
  for i in range(1,n+1):
    for j in range(n):
      if i + j == n:
        size.append([i, j])
  for k in range(len(size)):
    result = []
    for i in range(len(exp_multi[size[k][0]-1])):
      if size[k][1] != 0:
        for j in range(len(exp_divide[size[k][1]-1])):
          if expInvalid(exp_multi[size[k][0]-1][i], exp_divide[size[k][1]-1][j]):
            result.append(exp_multi[size[k][0]-1][i] + "/" + exp_divide[size[k][1]-1][j])
      else:
        result = exp_multi[size[k][0]-1]
    df = pd.DataFrame({"congthuc": result})
    df.to_csv(f"drive/MyDrive/sinh_loc_cong_thuc/congthuc/congthuc{size[k][0]}_{size[k][1]}.csv",index = False)
  return len(exp_multi[-1])
  
saveFile(10)
'''

In [ ]:
'''
#test file sinh ra
def split_exp(exp):
    if exp[0] in ["-", "+"]:
        pass
    else:
        exp = "+" + exp
    dataA = exp[1:].split("+")
    result = []
    index = 0
    for i in range(len(dataA)):
        dataB = dataA[i].split("-")
        for j in range(len(dataB)):
            result.append(dataB[j])
    for i in range(len(exp)):
        if exp[i] in ["+", "-"]:
            result[index] = exp[i] + result[index]
            index += 1
    return result

#process : hàm xử lí chuỗi(+a*e/e ==> +a)
def process(element):
    if len(element) == 2:
        return element
    else:
      
        dicter = {}
        dicter[element[1]] = 1
        op = element[0]
        the_dividend = ""
        divisor = ""
        for i in range(2, len(element)):
            if element[i] == "*":
                try:
                    dicter[element[i+1]] += 1
                except:
                    dicter[element[i+1]] = 1
            elif element[i] == "/":
                try:
                    dicter[element[i+1]] -= 1
                except:
                    dicter[element[i+1]] = -1

        for key, value in dicter.items():
            if value < 0:
                for i in range(-value):
                    divisor += key
            elif value > 0:
                for i in range(value):
                    the_dividend += key

        if len(divisor) == len(the_dividend) == 0:
            return op+"1"
        elif len(divisor) == 0:
            return op + "".join(sorted(list(the_dividend)))
        elif len(the_dividend) == 0:
            return op+"1"+"/"+divisor
        return op+"".join(sorted(list(the_dividend)))+"/" + "".join(sorted(list(divisor)))

#convert : chuyển một chuỗi thành cùng một kiểu
#exp : expression (biểu thức)
def convert(exp):
    data = split_exp(exp)
    data_process = []
    dicter = {}
    result = []
    for i in range(len(data)):
        data_process.append(process(data[i]))

    for i in range(len(data_process)):
        if data_process[i][0] == "+":
            try:
                dicter[data_process[i][1:]] += 1
            except:
                dicter[data_process[i][1:]] = 1
        else:
            try:
                dicter[data_process[i][1:]] -= 1
            except:
                dicter[data_process[i][1:]] = -1

    for key, value in dicter.items():
        if value == 0:
            pass
        else:
            result.append(str(value)+key)

    return "\\".join(sorted(result))

#clenData : Loại bỏ các công thức trùng nhau
#url : đường dẫn đến file, name_collum: tên cột

def cleanData(ex_data_df):
    result = []
    dicter = {}
    print("Số công thức: {}".format(len(ex_data_df)))

    for i in range(len(ex_data_df)):
        try:
            k = dicter[convert(ex_data_df[i])]
        except:
            dicter[convert(ex_data_df[i])] = 1
            result.append(ex_data_df[i])
    print("số công thức sau khi lọc: {}".format(len(result)))

    return "Done"
    
cleanData(list(pd.read_csv('drive/MyDrive/sinh_loc_cong_thuc/congthuc/9bien_0.csv')["congthuc"]))
'''
